<a href="https://colab.research.google.com/github/abdulrahman-hassanin/numbers_signs_recognition/blob/master/Number_signs_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K

Using TensorFlow backend.


# Load Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/My Drive/Colab Notebooks/Numbers Sign Dataset'

test_signs.h5  train_signs.h5


In [0]:
import h5py
def load_dataset():
  train_dataset = h5py.File('/content/drive/My Drive/Colab Notebooks/Numbers Sign Dataset/train_signs.h5', 'r')
  train_set_x_orig = np.array(train_dataset["train_set_x"][:])
  train_set_y_orig = np.array(train_dataset["train_set_y"][:])
  
  test_dataset = h5py.File('/content/drive/My Drive/Colab Notebooks/Numbers Sign Dataset/test_signs.h5', 'r')
  test_set_x_orig = np.array(test_dataset["test_set_x"][:])
  test_set_y_orig = np.array(test_dataset["test_set_y"][:])
  
  classes = np.array(test_dataset["list_classes"][:])
  
  train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
  test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
  
  return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [5]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalization X
X_train = X_train_orig / 255
X_test = X_test_orig / 255

# Conver Y into one_hot 
from keras.utils.np_utils import to_categorical
Y_train = np.eye(6)[Y_train_orig.reshape(-1)]
Y_test = np.eye(6)[Y_test_orig.reshape(-1)]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_test_orig.shape)
print(Y_test[0])

(1080, 64, 64, 3)
(120, 64, 64, 3)
(1080, 6)
(120, 6)
(1, 120)
[1. 0. 0. 0. 0. 0.]


# Build ResNet50 Model


### 1.Identity Block

In [0]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    
    #### main path
    
    # First component
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding="valid", name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second Component
    X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    # Third Component
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name= bn_name_base + '2c')(X)
    
    # Add shortcut
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X
    
    

### The convolutional block

In [0]:
def convolutional_block(X, f, filters, stage, block, s=2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component
    X = Conv2D(F3, (1, 1), strides = (1, 1), name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(F3, (1,1), strides = (s, s), name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    

    
    return X 

### ResNet50

In [0]:
def ResNet50(input_shape = (64, 64, 30), classes=6):
  
  X_input = Input(input_shape)
  
      
  # Zero-Padding
  X = ZeroPadding2D((3, 3))(X_input)

  # Stage 1
  X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1')(X)
  X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3, 3), strides=(2, 2))(X)

  # Stage 2
  X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
  X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
  X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

  # Stage 3 
  X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

  # Stage 4 
  X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

  # Stage 5 
  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='d')

  # AVGPOOL 
  X = AveragePooling2D(pool_size=(2,2),name = "avg_pool")(X)

  # output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)


  # Create model
  model = Model(inputs = X_input, outputs = X, name='ResNet50')

  return model


In [9]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

In [10]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, Y_train, epochs = 20, batch_size = 32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
1080/1080 [==============================] - 196s 182ms/step - loss: 3.6401 - acc: 0.2250
Epoch 2/20
1080/1080 [==============================] - 184s 170ms/step - loss: 2.0136 - acc: 0.4296
Epoch 3/20
1080/1080 [==============================] - 180s 167ms/step - loss: 0.8550 - acc: 0.7065
Epoch 4/20
1080/1080 [==============================] - 179s 166ms/step - loss: 0.8071 - acc: 0.7481
Epoch 5/20
1080/1080 [==============================] - 180s 167ms/step - loss: 0.4906 - acc: 0.8278
Epoch 6/20
1080/1080 [==============================] - 179s 166ms/step - loss: 0.2324 - acc: 0.9102
Epoch 7/20
1080/1080 [==============================] - 180s 166ms/step - loss: 0.1146 - acc: 0.9509
Epoch 8/20
1080/1080 [==============================] - 181s 167ms/step - loss: 0.1724 - acc: 0.9417
Epoch 9/20
1080/1080 [==============================] - 180s 166ms/step - loss: 0.1571 - acc: 0.95

In [0]:
model.save('Numbers_signs_20epochs'+'.h5')


In [16]:
preds = model.evaluate(X_test, Y_test)
print("loss = "+str(preds[0]))
print("acc = "+str(preds[1]))

120/120 [==============================] - 2s 21ms/step
loss = 0.2242736061414083
acc = 0.9416666706403096
